In [19]:
import pymongo as pmn
from sshtunnel import SSHTunnelForwarder
import pprint
import datetime
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np

In [20]:
MONGO_HOST = "10.2.81.40"
MONGO_DB = "VectorX_DWH"
MONGO_PORT = 65372

In [21]:
server = SSHTunnelForwarder(
    ssh_address_or_host=("10.2.81.40"),
    ssh_username="mmartinson",
    ssh_password="hyklapy",
    remote_bind_address=('127.0.0.1', 27017),
)

server.start()

client = pmn.MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
db = client[MONGO_DB]
pprint.pprint(db.collection_names())
server.stop()

['documents', 'topic_by_date']


In [17]:
server.stop()

In [22]:
server.start()
client = pmn.MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
db = client[MONGO_DB]

2018-03-16 17:45:52,653| ERROR   | Could not establish connection from ('127.0.0.1', 43528) to remote side of the tunnel


In [6]:
lvl1_names = set()
cursor = db.documents.find({}).limit(1000)
for i, row in tqdm(enumerate(cursor)):
    if len(row['topics']) > 0:
        lvl1_names.add(row['topics'][0]['lvl1']['id'])



0it [00:00, ?it/s]
723it [00:40, 18.06it/s]A
102it [00:12,  7.95it/s]
427it [00:12, 33.02it/s]
671it [00:13, 51.47it/s]
813it [00:17, 45.24it/s]
1000it [00:18, 55.53it/s]

In [7]:
lvl1_names

{'lvl1_бизнес',
 'lvl1_культура',
 'lvl1_медиа',
 'lvl1_наука_и_техника',
 'lvl1_общество_и_путешествия',
 'lvl1_политика',
 'lvl1_происшествия',
 'lvl1_силовые_структуры',
 'lvl1_спорт',
 'lvl1_экономика_и_финансы'}

In [8]:
lvl1 = pd.DataFrame(columns=list(lvl1_names))
cur_time = datetime.datetime(2017, 8, 1, 0, 0, 0)
delta = datetime.timedelta(hours=1)
defoult_nicht = {
  'lvl1_бизнес' : 0,
 'lvl1_культура' : 0,
 'lvl1_медиа' : 0,
 'lvl1_наука_и_техника' : 0,
 'lvl1_общество_и_путешествия' : 0,
 'lvl1_политика' : 0,
 'lvl1_происшествия' : 0,
 'lvl1_силовые_структуры' : 0,
 'lvl1_спорт' : 0,
 'lvl1_экономика_и_финансы' : 0}
lvl1 = lvl1.append(defoult_nicht, ignore_index=True)
lvl1

,lvl1_общество_и_путешествия,lvl1_экономика_и_финансы,lvl1_бизнес,lvl1_силовые_структуры,lvl1_наука_и_техника,lvl1_спорт,lvl1_медиа,lvl1_политика,lvl1_происшествия,lvl1_культура
0,0,0,0,0,0,0,0,0,0,0


In [9]:
cursor = db.documents.find({})
for row in tqdm(cursor):
    if (row['rss_pubDate'] > cur_time + delta):
        cur_time = cur_time + delta
        #lvl1.iloc[-1] /= sum(lvl1.iloc[-1])
        lvl1 = lvl1.append(defoult_nicht, ignore_index=True)

    if len(row['topics']) <= 0:
        continue
        
    lvl1.iloc[-1][row['topics'][0]['lvl1']['id']] += row['topics'][0]['probability']

IndexError: Cursor instances do not support slice steps

In [ ]:
lvl3_art_word = pd.DataFrame()

start_time = datetime.datetime(2017, 8, 1, 0, 0, 0)
delta = datetime.timedelta(hours=2)

cur_time = 0
while start_time + delta < datetime.datetime(2017, 9, 1, 0, 0, 0):
    cursor = db.documents.find({"rss_pubDate" : {'$gt' : start_time, '$lt' : start_time + delta}}).limit(100)
    
    for row in cursor:
        if len(row['topics']) > 0:
            if len(row['topics'][0]['lvl3']['distribution']) >= 3:
                lvl3_art_word.append(row['topics'][0]['lvl3'], ignore_index=True)
    start_time += delta
    cur_time += 1
    print(cur_time, end=' ')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 

In [33]:
num = np.array(num)

print(num.mean(), num.min(), num.max())

print(num.shape)

85.55263157894737 0 96
(342,)


In [ ]:
distr = pd.DataFrame()
start_time = datetime.datetime(2017, 8, 1, 0, 0, 0)
delta = datetime.timedelta(hours=2)

while start_time + delta < datetime.datetime(2017, 9, 1, 0, 0, 0):
    
    #cursor = db.documents.find({"rss_pubDate" : {'$gt' : start_time, '$lt' : start_time + delta}}).limit(100)
    cursor = db.documents.aggregate([ {"$group" : {""}, "$count" : "number"}])
    num.append(0)
    
    for row in cursor:
        
        if len(row['topics']) > 0:
            if len(row['topics'][0]['lvl1']['distribution']) >= 5:
                num[-1] += 1
    start_time += delta